База данных для хранения исходных данных, промежуточных и окончательных результатов.


In [1]:
import sqlite3
from conllu import parse
from os import listdir

In [2]:
conn = sqlite3.connect('DoctorWho.db')
cur = conn.cursor()

In [3]:
cur.executescript('''
CREATE TABLE Sentences (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Sentence TEXT,
    Author TEXT,
    Name TEXT,
    Link TEXT
);

CREATE TABLE Words (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Token TEXT,
    Lemma TEXT,
    POS TEXT,
    ID_Sent INTEGER,
    FOREIGN KEY (ID_Sent) REFERENCES Sentences(ID)
);
''')
conn.commit()

Для коллекции предложений conllu идёт первичная итерация по предложениям sent с метадатой meta, вторичная итерация по токенам token с леммой lemma и тегом tag

In [4]:
# пишем данные в базу
id_sent = 1
id_token = 1
for file in listdir('fics_conllus'):
    filename = 'fics_conllus/' + file
    with open(filename) as f:
        text = f.read()
    tokens = parse(text)
    for line in tokens:
        meta = line.metadata['metadata']
        ind1 = meta.find('автор')
        ind2 = meta.find('ссылка')
        name = meta[10:ind1].rstrip(', ')
        author = meta[ind1 + 7 : ind2].rstrip(', ')
        link = meta[ind2 + 8:]
        sent = str(line.metadata['text'])
        cur.execute("INSERT INTO Sentences VALUES (?,?,?,?,?)", (id_sent, sent, author, name, link)) # id, предложение, метадата
        for word in line:
            token = word['form'].lower()
            lemma = word['lemma'].lower()
            tag = word['upos']
            if tag != 'PUNCT':
                cur.execute(
                    """
                        INSERT INTO Words
                        VALUES (?,?,?,?,?)
                    """, (id_token,token,lemma,tag,id_sent)) # id, токен, лемма, тег, id предложения, из которго взято слово
                id_token +=1
        id_sent += 1
conn.commit()

In [5]:
# Создаём индексы, чтобы ускорить поиск

cur.executescript('''
CREATE INDEX idx_token ON Words(Token);
CREATE INDEX idx_lemma ON Words(Lemma);
CREATE INDEX idx_pos ON Words(POS);
''')
conn.commit()

In [6]:
conn.close()